In [225]:
class Tokenizer():
    def __init__(self):
        self.word_dict = {'oov': 0}
        self.fit_checker = False
  
    def preprocessing(self, sequences):
        result = []
        sequences=str(sequences) # 소문로 변환하기 위하여 str처리
        sequences=sequences.lower()
        sequences=sequences.split(',') #입력 받은 문장끼리 분류하는 과정
        for i in sequences:
            i=i.replace('.','').replace('!','').replace(" '",'').replace("'",'').replace('[','').replace(']','').replace('?','') #특수문자제거
            i=i.split(' ') #white space 단위로 토큰화
            result.append(i) #nested list 형태로 만듬
        return result
  
    def fit(self, sequences):
        self.fit_checker = False
        tokens = self.preprocessing(sequences)
        to=[]
        for i in tokens:
            to.extend(i)  #nested list를 합침

        to=dict.fromkeys(to)  #중복을 제거
        to=list(to)
        for j,i in enumerate(to): #word_dict에 토큰을 넣음
            self.word_dict[i]=j+1 #기존에 0이 있어서 1씩 추가해서 넣음
        self.fit_checker = True
    def transform(self, sequences):
        result = []
        tokens= self.preprocessing(sequences)
        if self.fit_checker: 
            for item in tokens:  #nested list중 첫번째 list에 접근
                for i,j in enumerate(item): #첫번째 list의 원소와 index를 받음
                    if j in self.word_dict.keys(): #원소가 word_dict에 있다면 해당 원소를 word_dict의 index로 변환
                        item[i]=self.word_dict[j]
                    if j not in self.word_dict.keys(): #원소가 word_dict에 없다면 해당 원소를 'oov'의 index로 변환
                        item[i]=self.word_dict['oov']
                result.append(item) #nested list로 만들어주기 위함
            return result
        else:
            raise Exception("Tokenizer instance is not fitted yet.")
    def fit_transform(self, sequences):
        self.fit(sequences)
        result = self.transform(sequences)
        return result


    

In [226]:
a=Tokenizer()
sen=['I want to go to school','I like Pizza!','I go to school.']
a.preprocessing(sen)
a.fit(sen)
a.transform(sen)

[[1, 2, 3, 4, 3, 5], [1, 6, 7], [1, 4, 3, 5]]

In [227]:
class TfidfVectorizer:
    def __init__(self,tokenizer):
        self.tokenizer = tokenizer
        self.fit_checker = False
    def fit(self, sequences):
        tokenized = self.tokenizer.fit_transform(sequences)
        import math
        b=[]
        c=[]
        d=[]
        e=[]
        for to in tokenized: #각 list내의 중복을 제거하기 위함
            to=dict.fromkeys(to)
            to=list(to)
            b.append(to)
        for i in b:
            c.extend(i) #nested list를 풀어주는 과정
        d=dict.fromkeys(c) #각 토큰별로 idf값을 구해야하므로 중복을 제거 
        d=list(d)
        for i in d:
            a=math.log(len(tokenized)/(1+c.count(i))) #각 토큰이 들어있는 문장의 개수를 구해야 하므로 list내의 중복을 제거한 c에서 카운트
            e.append(a)
        self.d=d
        self.e=e
        self.fit_checker = True
    def transform(self, sequences):
        if self.fit_checker:
            tokenized = self.tokenizer.transform(sequences)   
            to=self.e   
            d=self.d                
            self.tfidf_matrix=[]
            for item in tokenized: # 특정 문장에 토큰이 나타나는 횟수를 구하기 위해 tokenized에서 for 문 사용 
                c=[] #nested list를 만들기 위함
                for i,j in enumerate(d):  #[[1, 2, 3, 4, 3, 5], [1, 6, 7], [1, 4, 3, 5]] 인 경우 한 list에서 3이 두번 나오므로  
                    a=item.count(j)*to[i] #이를 고려하여 tokenized의 리스트에서 카운트
                    c.append(a)
                self.tfidf_matrix.append(c)
            return self.tfidf_matrix
        else:
            raise Exception("TfidfVectorizer instance is not fitted yet.")
    def fit_transform(self, sequences):
        self.fit(sequences)
        return self.transform(sequences)

In [229]:
a=Tokenizer()
b=TfidfVectorizer(a)

sen=['I want to go to school','I like Pizza!','I go to school.']
b.fit(sen)
b.transform(sen)
pd.DataFrame(b.transform(sen),columns=list(b.tokenizer.word_dict.keys())[1:])

,i,want,to,go,school,like,pizza
0,-0.287682,0.405465,0.0,0.0,0.0,0.000000,0.000000
1,-0.287682,0.000000,0.0,0.0,0.0,0.405465,0.405465
2,-0.287682,0.000000,0.0,0.0,0.0,0.000000,0.000000
